# Computing observables in the FONLL scheme

The actual implementation of FONLL observables requires 
the computation of the partonic coefficients functions using different
FFNS settings with `yadism` and the possibility to compute coexisting flavor number PDFs, 
which is provided by the `EKO` library.
In this tutorial we explain, using an example, how observables can be calculated using a fixed input PDF by evolving it to the three different flavor number schemes.

As a first step we define the define the inputs needed for a computation of a DIS observable. 
In the following example we will compute predictions for $F_{2c}$.

In [55]:
# Define a DIS observable runcard:
observablecard = {
    # Process type: "EM", "NC", "CC"
    "prDIS": "NC",
    # Projectile: "electron", "positron", "neutrino", "antineutrino"
    "ProjectileDIS": "electron",
    # Scattering target: "proton", "neutron", "isoscalar", "lead", "iron", "neon" or "marble"
    "TargetDIS": "proton",
    # Interpolation: if True use log interpolation
    "interpolation_is_log": True,
    # Interpolation: polynomial degree, 1 = linear, ...
    "interpolation_polynomial_degree": 4,
    # Interpolation: xgrid values
    # Note: for illustrative purposes the grid is chosen very small here
    "interpolation_xgrid": [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
    # Observables configurations
    "observables": {
      "F2_charm": [
        {
            "y": 0.,
            "x": 1e-4,
            "Q2": 15.0**2,
        },
        {
            "y": 0.,
            "x": 1e-3,
            "Q2": 15.0**2,
        },
        # Add here the kinematics of other datapoints
      ],
      # Potentially include observables other than F2_charm,
      # each of them has to be: TYPE_heaviness, where heaviness can take:
      # "charm", "bottom", "total" or "light".
    },
    # Projectile polarization faction, float from 0 to 1.
    "PolarizationDIS": 0.0,
    # Exchanged boson propagator correction
    "PropagatorCorrection": 0.0,
    # Restrict boson coupling to a single parton ? Monte Carlo PID or None for all partons
    "NCPositivityCharge": None,
}

# Define the theory parameters:
theorycard = {
    "CKM": "0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152", # CKM matrix elements
    "GF": 1.1663787e-05, # [GeV^-2] Fermi coupling constant
    "MP": 0.938, # [GeV] proton mass
    "MW": 80.398, # [GeV] W boson mass
    "MZ": 91.1876, # [GeV] Z boson mass
    "mc": 1.51, # [GeV] charm mass
    "mb": 4.92, # [GeV] bottom mass
    "mt": 172.5, # [GeV] top mass
    "kcThr": 1.0, # ratio of the charm matching scale over the charm mass
    "kbThr": 1.0, # ratio of the bottom matching scale over the bottom mass
    "ktThr": 1.0, # ratio of the top matching scale over the top mass
    "alphas": 0.118, # alphas value at the reference scale
    "Q0": 1.65, # [GeV] reference scale for the flavor patch determination
    "nf0": 4, # number of active flavors at the Q0 reference scale
    "PTO": 1, # perturbative order in alpha_s: 0 = LO (alpha_s^0), 1 = NLO (alpha_s^1) ...
    "alphaqed": 0.007496252, # alpha_em value
    "Qref": 91.2, # [GeV] reference scale for the alphas value
    "nfref": 5, # number of active flavors at the reference scale Qref
    "QED": 0, # QED correction to running of strong coupling: 0 = disabled, 1 = allowed
    "XIF": 1.0, # ratio of factorization scale over the hard scattering scale
    "XIR": 1.0, # ratio of renormalization scale over the hard scattering scale
    "TMC": 1, # include target mass corrections: 0 = disabled, 1 = leading twist, 2 = higher twist approximated, 3 = higher twist exact
    "n3lo_cf_variation": 0, # N3LO coefficient functions variation: -1 = lower bound, 0 = central , 1 = upper bound
    "MaxNfAs": 5, # maximum number of flavors in running of strong coupling
    "HQ": "POLE", # heavy quark mass scheme (only for DGLAP evolution)
    "MaxNfPdf": 5, # maximum number of flavors in running of PDFs (needed by pineko)
    "IC": 1, # 0 = perturbative charm only, 1 = intrinsic charm allowed
    "ModEv": "EXA",  # alphas evolution method, "EXA" = "exact", "TRN" = "truncated"
}

# Define the PDF evolution grid and other DGLAP evolution parameters
operatorcard = {
    # Evolution starting scale
    'mu0': theorycard["Q0"],
    # Evolution final scales: add here all the points needed in the evolved PDF
    # with the corresponding active number of flavor (Q_i, nf_i)
    # (we will update this entry later)
    'mugrid': [(10.0, 5),(15.0, 5),(20.0, 5),],
    # specify an interpolation grid
    'xgrid': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
    # DGLAP evolution settings
    'configs': {
        # evolution method: 'iterate-exact', 'iterate-expanded', 'truncated' ... see https://eko.readthedocs.io/en/latest/theory/DGLAP.html
        'evolution_method': 'iterate-exact',
        # other evolution parameters. See https://eko.readthedocs.io/en/latest/theory/DGLAP.html
        'ev_op_max_order': [10, 0],
        'ev_op_iterations': 10,
        # Interpolation: polynomial degree, 1 = linear, ...
        'interpolation_polynomial_degree': 4,
        # Interpolation: if True use log interpolation
        'interpolation_is_log': True,
        # Scale variations: "None", "expanded", "exponentiated"
        'scvar_method': None,
        # Backward evolution matching inversion: "None", "expanded", "exact"
        'inversion_method': None,
        # Number of parallel cores to use: 0 = all, 1 = one, -1 = all but one ...
        'n_integration_cores': 1,
        # Polarized evolution ?
        'polarized': False,
        # Time like evolution ?
        'time_like': False
    },
    # Debug options
    'debug': {
        'skip_singlet': False,
        'skip_non_singlet': False
    }
}

Then we compute the necessary coexisting flavor number PDFs starting from a given PDF defined for some value of $Q$ and corresponding $n_f$
using `EKO` (for more detailed information on the use of `EKO` we refer to the [documentation](https://eko.readthedocs.io) ).
Here, we show how to evolve `NNPDF40_nnlo_as_01180` to produce three PDFs in FFNS3, FFNS4, and FFNS5 respectively.

In [44]:
import lhapdf
import pathlib
from eko.io import runcards
from ekobox.evol_pdf import evolve_pdfs


# Define the PDF for which we want to make the prediction
# Here we take the replica 0 of NNPDF4.0 - remember to have it available via LHAPDF.
# You can install it if necessary via `lhapdf get NNPDF40_nnlo_as_01180`.
input_pdf = lhapdf.mkPDF("NNPDF40_nnlo_as_01180", 0)

# Build eko-like runcards objects
theory_card = runcards.Legacy(theorycard, {}).new_theory
operators_card = runcards.OperatorCard.from_dict(operatorcard)

# Iterate the flavor numbers
for nf in [3, 4, 5]:
    # Update the nf value in the mugrid
    operators_card.mugrid = [(mu, nf) for (mu,_) in operators_card.mugrid]

    eko_name = f"eko_nf{nf}.tar"
    pathlib.Path(eko_name).unlink(missing_ok=True)
    # Evolve the LHAPDF grid and store the output
    evolve_pdfs(
        initial_PDF_list=[input_pdf],
        theory_card=theory_card,
        operators_card=operators_card,
        store_path=f"eko_nf{nf}.tar",  # where to store the eko
        install=True, # Install in the evolved PDF in the LHAPDF directory
        name=f"NNPDF40_evolved_nf{nf}",  # name of the evolved pdf
    )

LHAPDF 6.5.0 loading /Users/giacomomagni/.conda/envs/yad_dev/share/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331100


[14:04:47] INFO     Interpolation: number of points = 8, polynomial degree = 4, logarithmic =  ]8;id=377600;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=48399;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/interpolation.py#538\538]8;;\
                    True                                                                                           

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=2.7224999999999997 ]8;id=390096;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=162837;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\
                    @ 4                                                                                            

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=135503;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=930612;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=448046;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=627029;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

...

           INFO     Evolution: Total time 2.412126 s                                                ]8;id=843737;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/evolution_operator/__init__.py\__init__.py]8;;\:]8;id=197850;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/evolution_operator/__init__.py#982\982]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=92553;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=599824;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=466736;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=998649;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

Overwriting old PDF installation


install_pdf NNPDF40_evolved_nf5


Next, we compute the coefficient functions using `yadism` and convolve them with the three PDFs generated in the script above. 
The example script works for `FONLL-A` and `FONLL-C` without damping. 
`FONLL-B` requires more care as one has to compute the same coefficient functions but at different perturbative orders.
Also note that the script below just combines various FFNSs, but does not produce a full VFNS.
For example, one might want to introduce here a dumping function to turn off e.g. $n_f=4$ contributions 
below the charm matching scale.

In [53]:
import yadism

# Define the settings for the different contributions to the FONLL observable
settings_list = [
    ["FONLL-FFNS", 3, "full", "NNPDF40_evolved_nf3"],
    ["FONLL-FFN0", 3, "full", "NNPDF40_evolved_nf3"],
    ["FONLL-FFNS", 4, "massless", "NNPDF40_evolved_nf4"],
    ["FONLL-FFN0", 4, "full", "NNPDF40_evolved_nf4"],
    ["FONLL-FFNS", 5, "full", "NNPDF40_evolved_nf5"],
]

single_scheme_result = []
# Iterate parts
for fns, nf, fonllparts, pdfname in settings_list:
    # Replace the relevant settings in the theory card
    theorycard["FNS"] = fns
    theorycard["NfFF"] = nf
    theorycard["FONLLParts"] = fonllparts
    pdf = lhapdf.mkPDF(pdfname)

    # Compute the observable in each of the schemes defined in settings_list
    out = yadism.run_yadism(theorycard, observablecard)

    # Store the results in single_scheme_result
    single_scheme_result.append(out.apply_pdf(pdf))

# Extract the different terms form single_scheme_result and combine them to
# obtain the coexisting flavor number scheme FONLL result
result = {}
for observable_name, kinematic_points in single_scheme_result[0].items():
    result[observable_name] = []
    for i in range(len(kinematic_points)):
        result[observable_name].append(
            single_scheme_result[0][observable_name][i]["result"]  # FONLL-FFNS3
            - single_scheme_result[1][observable_name][i]["result"]  # FONLL-FFN03
            + single_scheme_result[2][observable_name][i]["result"]  # FONLL-FFNS4
            - single_scheme_result[3][observable_name][i]["result"]  # FONLL-FFN04
            + single_scheme_result[4][observable_name][i]["result"]  # FONLL-FFNS5
        )

LHAPDF 6.5.0 loading /Users/giacomomagni/.conda/envs/yad_dev/share/LHAPDF/NNPDF40_evolved_nf3/NNPDF40_evolved_nf3_0000.dat
NNPDF40_evolved_nf3 PDF set, member #0, version 1


[14:08:27] INFO     Interpolation: number of points = 8, polynomial degree = 4, logarithmic =  ]8;id=422517;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/interpolation.py\interpolation.py]8;;\:]8;id=211907;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/interpolation.py#538\538]8;;\
                    True                                                                                           

           INFO     {'MZ2': 8315.178393760001, 'CKM': CKM([[9.49221518e-01 5.07600900e-02  ]8;id=560823;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/coefficient_functions/coupling_constants.py\coupling_constants.py]8;;\:]8;id=152965;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/coefficient_functions/coupling_constants.py#53\53]8;;\
                    1.20409000e-05] [5.07150400e-02 9.47604903e-01 1.68100000e-03]                                 
                    [7.43044000e-05 1.62409000e-03 9.98304719e-01]]), 'sin2theta_weak':                            
                    0.23121, 'MW2': 6463.838403999999}                                                             

           INFO     {'process': 'NC', 'projectilePID': 11, 'polarization': 0.0,            ]8;id=718434;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/coefficient_functions/coupling_constants.py\coupling_constants.py]8;;\:]8;id=314592;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/coefficient_functions/coupling_constants.py#54\54]8;;\
                    'propagatorCorrection': 0.0, 'nc_pos_charge': None}                                            

           INFO     RenScaleVar: True, FactScaleVar: True                                    ]8;id=893716;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=78787;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/esf/scale_variations.py#62\62]8;;\

...

           INFO     computing P_qg_0 - took: 0.023234 s                                      ]8;id=371523;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=692913;file:///Volumes/Git_Workspace/physicstools/NN3PDF/yadism/src/yadism/esf/scale_variations.py#85\85]8;;\
Starting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

took 0.22 s

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=929556;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=311260;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=486822;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=754730;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=708339;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=81565;file:///Users/giacomomagni/.conda/envs/yad_dev/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

In [54]:
result

{'F2_charm': [1036.7275411503838, -303.19449001078374]}